In [288]:
import pandas as pd

In [289]:

df = pd.read_excel("casagrands.xls")

In [290]:
# df=df[["LastName","FirstName","Email"]]

In [291]:
text_cols = df.select_dtypes(include="object").columns
df[text_cols] = df[text_cols].fillna("")

In [292]:
import re
for col in ['FirstName', 'LastName']:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace(r'[^a-zA-Z. ]', '', regex=True)



In [293]:
df.head(15)

,LastName,FirstName,Salutation,Name,Company,Phone,MobilePhone,Email,SYS_Mobile__c,WA_Formatted_Phone__c,...,Mobile8__c,Mobile9__c,Mobile10__c,Country_Code__c,First_Phone__c,Second_Phone__c,Third_Phone__c,Fourth_Phone__c,Sixth_Phone__c,Seventh_Phone__c
0,As Chakravarthi,,,As Chakravarthi,As Chakravarthi,919962154351,9.199622e+11,aschakrav@gmail.com,9.199622e+11,9.199622e+11,...,NaN,NaN,NaN,91.0,9962154351,9.080857e+09,8.778092e+09,NaN,NaN,NaN
1,Raveendra,Meda,,Meda Raveendra,Meda Raveendra,919962041716,9.199620e+11,mrreddy@ramcadds.in,9.199620e+11,9.199620e+11,...,NaN,NaN,NaN,91.0,9962041716,NaN,NaN,NaN,NaN,NaN
2,Kannan,,Mr,Kannan,Kannan,918189923399,9.181899e+11,kannan.ican1891@gmail.com,9.181899e+11,9.181899e+11,...,NaN,NaN,NaN,91.0,8189923399,NaN,NaN,NaN,NaN,NaN
3,Ramyaa,,Mr,Ramyaa,Ramyaa,919962746446,9.199627e+11,ramyaa.doc@gmail.com,9.199627e+11,9.199627e+11,...,NaN,NaN,NaN,91.0,9962746446,9.003088e+09,NaN,NaN,NaN,NaN
4,Vasan.K,,Mr,Vasan.K,Vasan.K,918939149522,9.189391e+11,vasankm@yahoo.com,9.189391e+11,9.189391e+11,...,NaN,NaN,NaN,91.0,8939149522,NaN,NaN,NaN,NaN,NaN
5,sundar,,Mr,sundar,sundar,919962502711,9.199625e+11,ksundar2001@gmail.com,9.199625e+11,9.199625e+11,...,NaN,NaN,NaN,91.0,9962502711,NaN,NaN,NaN,NaN,NaN
6,Pandiyan,,Mr,Pandiyan,Pandiyan,919790947384,9.197909e+11,acp3012@gmail.com,9.197909e+11,9.197909e+11,...,NaN,NaN,NaN,91.0,9790947384,NaN,NaN,NaN,NaN,NaN
7,K.Uma,,Mr,K.Uma,K.Uma,919840891528,9.198409e+11,kuma189@yahoo.com,9.198409e+11,9.198409e+11,...,NaN,NaN,NaN,91.0,9840891528,NaN,NaN,NaN,NaN,NaN
8,Vinodh Kumar,,,Vinodh Kumar,Vinodh Kumar,919940194192,9.199402e+11,vinoths99401@gmail.com,9.199402e+11,9.199402e+11,...,NaN,NaN,NaN,91.0,9940194192,NaN,NaN,NaN,NaN,NaN
9,Soundar,,,Soundar,Soundar,919677023477,9.196770e+11,phyraj2008@gmail.com,9.196770e+11,9.196770e+11,...,NaN,NaN,NaN,91.0,9677023477,NaN,NaN,NaN,NaN,NaN


In [294]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NameValidation") \
    .getOrCreate()


In [295]:

spark_df = spark.createDataFrame(df)


In [296]:
from pyspark.sql import functions as F

spark_df = spark_df.withColumn(
    "full_name",
    F.trim(F.concat_ws(" ", F.col("FirstName"), F.col("LastName")))
)


In [297]:
spark_df.show(truncate=False)



+---------------+---------------+----------+---------------------------+---------------------------+------------+----------------+-------------------------------+----------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+---------------------------+
|LastName       |FirstName      |Salutation|Name                       |Company                    |Phone       |MobilePhone     |Email                          |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Second_Phone__c|Third_Phone__c|Fourth_Phone__c|Sixth_Phone__c|Seve

In [298]:


import pyspark.sql.functions as F

spark_df = (
    spark_df

    # ------------------------------------------------------------------
    # 0. Name not present (highest priority)
    # ------------------------------------------------------------------
    .withColumn(
        "name_not_present",
        F.when(
            F.col("full_name").isNull() |
            (F.trim(F.col("full_name")) == ""),
            True
        )
    )

    # ------------------------------------------------------------------
    # 1. Only dots (ONLY if name is present)
    # ------------------------------------------------------------------
    .withColumn(
        "only_dots_name",
        F.when(
            F.col("name_not_present").isNull() &
            F.col("full_name").rlike(r"^\s*\.{1,}\s*$"),
            True
        )
    )

    # ------------------------------------------------------------------
    # 2. Only single initial (ONLY if name is present)
    # ------------------------------------------------------------------
    .withColumn(
        "only_initial_name",
        F.when(
            F.col("name_not_present").isNull() &
            F.col("full_name").rlike(r"^\s*\.?[A-Za-z]\.?\s*$"),
            True
        )
    )

    # ------------------------------------------------------------------
    # 3. Invalid / blocking names (ONLY if name is present)
    # ------------------------------------------------------------------
    .withColumn(
        "invalid_name",
        F.when(
            F.col("name_not_present").isNull() &
            F.col("full_name").rlike(
                r"(?i)\b(do not call|dont call|no call|remove|xyz|abcd|abc|test)\b"
            ),
            True
        )
    )


    .withColumn(
        "valid_names",
        F.when(
            F.col("name_not_present").isNotNull() |
            F.col("only_dots_name").isNotNull() |
            F.col("only_initial_name").isNotNull() |
            F.col("invalid_name").isNotNull(),
            False
        )
        .otherwise(None)
    )
)



In [299]:
spark_df.show(161,truncate=False)


+-------------------+------------------------+----------+---------------------------------------------+---------------------------------------------+------------+----------------+-----------------------------------+----------------+---------------------+-----------------------------------+--------------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+--------------------------------------------+----------------+--------------+-----------------+------------+-----------+
|LastName           |FirstName               |Salutation|Name                                         |Company                                      |Phone       |MobilePhone     |Email                              |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                           |Encrypted_SMobile__c|Mobile_Phone__c |P

In [300]:

# spark_df = spark_df.drop(
#     "invalid_names",
#     "is_valid_names"
# )

In [301]:
from pyspark.sql import functions as F

spark_df = (
    spark_df
    # convert FirstName and LastName to lowercase
    .withColumn("FirstName", F.lower(F.col("FirstName")))
    .withColumn("LastName", F.lower(F.col("LastName")))

    # drop full_name column
    .drop("full_name")
)


In [302]:
from pyspark.sql.functions import col, trim, when

EMAIL_REGEX = (
    r'^(?=.{1,254}$)(?=.{1,64}@)'
    r'[A-Za-z0-9_%+-]+(\.[A-Za-z0-9_%+-]+)*'
    r'@'
    r'[A-Za-z0-9-]+(\.[A-Za-z0-9-]+)*\.[A-Za-z]{2,}$'
)

spark_df = spark_df.withColumn(
    "is_valid_email",
    when(col("Email").isNull(), False)  # Missing email → False
    .when(~trim(col("Email")).rlike(EMAIL_REGEX), False)  # Invalid email → False
    .otherwise(None)  # Valid email → blank/NULL
)


In [303]:
spark_df.show(truncate=False)


+---------------+---------------+----------+---------------------------+---------------------------+------------+----------------+-------------------------------+----------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+----------------+--------------+-----------------+------------+-----------+--------------+
|LastName       |FirstName      |Salutation|Name                       |Company                    |Phone       |MobilePhone     |Email                          |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone__c |Phone__c        |Mobile3__c     |Mobile4__c|Mobile5__c|Mobile6__c|Mobile7__c|Mobile8__c|Mobile9__c|Mobile10__c|Country_Code__c|First_Phone__c|Seco

In [304]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# --------------------------------------------------
# Preserve original order
# --------------------------------------------------
spark_df = spark_df.withColumn("_row_order", F.monotonically_increasing_id())


spark_df = spark_df.withColumn(
    "email_not_present",
    F.when(
        F.col("Email").isNull() |
        (F.trim(F.col("Email")) == "") |
        (F.lower(F.trim(F.col("Email"))) == "nan"),
        True
    )
)


spark_df = spark_df.withColumn(
    "invalid_email_format",
    F.when(
        (F.col("email_not_present").isNull()) &  # only if email exists
        (F.col("is_valid_email") == False),
        True
    )
)
email_window = Window.partitionBy(F.lower(F.trim(F.col("Email"))))

spark_df = spark_df.withColumn(
    "duplicate_email",
    F.when(
        (F.col("email_not_present").isNull()) &           # email exists
        (F.col("invalid_email_format").isNull()) &        # email valid
        (F.col("is_valid_email").isNull()) &              # email valid
        (F.count("*").over(email_window) > 1),           # duplicate
        True
    )
)

spark_df = spark_df.withColumn(
    "valid_email",
    F.when(
        (F.col("email_not_present") == True) |
        (F.col("invalid_email_format") == True) |
        (F.col("duplicate_email") == True),
        False
    )
)

# --------------------------------------------------
# Restore original order
# --------------------------------------------------
spark_df = spark_df.orderBy("_row_order").drop("_row_order")


In [305]:

spark_df=spark_df.drop("is_valid_email")

In [306]:
spark_df.show(100,truncate=False)


+-------------------+------------------------+----------+---------------------------------------------+---------------------------------------------+------------+----------------+-------------------------------+----------------+---------------------+-------------------------------+--------------------+----------------+----------------+---------------+----------+----------+----------+----------+----------+----------+-----------+---------------+--------------+---------------+--------------+---------------+--------------+----------------+----------------+--------------+-----------------+------------+-----------+-----------------+--------------------+---------------+-----------+
|LastName           |FirstName               |Salutation|Name                                         |Company                                      |Phone       |MobilePhone     |Email                          |SYS_Mobile__c   |WA_Formatted_Phone__c|Email__c                       |Encrypted_SMobile__c|Mobile_Phone_

In [307]:
df = spark_df.toPandas()


In [308]:
df.to_csv("output5.csv", index=False)


In [153]:
df.to_csv(r"C:\Users\Vasala\Desktop\datasets\lead_data.csv", index=False)
